In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from datetime import datetime as dt
from tqdm.notebook import tqdm
import pickle
import datetime
import warnings
from IPython.display import clear_output
warnings.filterwarnings("ignore")

#Read in options data
data = pd.read_csv('data/2010-2023FILTERED_MOREDELTA.csv').drop(columns='Unnamed: 0')

#Read in SPY data
spy = pd.read_csv('data/SPY.csv')
spy = spy.set_index('Date')

In [2]:
def GetIVS(df):

    count = 0
    #Organize data by expiry date and strike date
    print('DATA ORGANIZATION PROGRESS')
    organized_data = {}
    for row in range(len(df)):
        if count % 100000 == 0:
            clear_output()
            print('DATA ORGANIZATION PROGRESS')
            print(count)
        count += 1
        exp_str = str(df.loc[row]['EXPIRY_DATE'])[1:]
        strike = float(df.loc[row]['STRIKE'])
        rowData = df.iloc[row]
        if exp_str not in organized_data.keys():
            organized_data[exp_str] = {}        
        if strike not in organized_data[exp_str].keys():
            organized_data[exp_str][strike] = []
        
        organized_data[exp_str][strike].append(rowData)

    count = 0
    #Make the data into dataframes
    print('CREATING DATAFRAMES')
    sorted_data = {}
    for exp in organized_data:
        if count % 50 == 0:
            clear_output()
            print('CREATING DATAFRAMES')
            print(count)
        count += 1
        sorted_data[exp] = {}
        for strike in organized_data[exp]:
            sorted_data[exp][strike] = pd.DataFrame(organized_data[exp][strike])
            
    count = 0
    #Create empty IVS
    print("CREATING EMPTY IVS")
    observed_days = {}
    for exp in sorted_data.keys():
        if count % 50 == 0:
            clear_output()
            print('CREATING EMPTY IVS')
            print(count)
        count += 1
        for strike in sorted_data[exp].keys():
            for date in sorted_data[exp][strike]['QUOTE DATE']:
                date = date[1:]
                if date not in observed_days.keys():
                    observed_days[date] = pd.DataFrame()

    count = 0
    #Populate IVS
    print("POPULATING IVS")
    ivs = observed_days.copy()
    for exp in sorted_data.keys():
        if count % 50 == 0:
            clear_output()
            print('POPULATING IVS')
            print(count)
        count += 1
        for strike in sorted_data[exp].keys():
            for row in range(len(sorted_data[exp][strike])):
                date = sorted_data[exp][strike].iloc[row]['QUOTE DATE'][1:]
                dte = sorted_data[exp][strike].iloc[row]['DTE']
                moneyness = strike 
                iv = sorted_data[exp][strike].iloc[row]['IV']
                ivs[date].loc[dte, moneyness] = iv

    count = 0
    #Create Moneyness Grid
    print('CREATING MONEYNESS GRID')
    final_ivs = {}
    for date in ivs:
        if count % 50 == 0:
            clear_output()
            print('CREATING MONEYNESS GRID')
            print(count)
        count += 1
        if date in spy.index:
            spy_price = spy.loc[date]['Close']
            moneyness_list = []
            dte_month_list = []
            strikes = list(ivs[date].columns)
            dtes = list(ivs[date].index)
            for dte in dtes:
                dte_month_list.append((dte / 365.0))
            for strike in strikes:
                moneyness = strike/spy_price
                moneyness_list.append(moneyness)
            final_ivs[date] = ivs[date].copy(deep=True)
            final_ivs[date].columns = moneyness_list
            final_ivs[date].index = dte_month_list
            final_ivs[date] = final_ivs[date].sort_index().sort_index(axis=1)

    return final_ivs

In [3]:
ivs = GetIVS(data)

with open('data/R1_10-23_IVS_MOREDELTA_ANN_ADJ.pkl', 'wb') as file:
    pickle.dump(ivs, file)

CREATING MONEYNESS GRID
3400
